# 1. Tokenization

## Libraries

In [2]:
import re
import numpy as np
import pandas as pd
import collections as cl
import nltk
from nltk.lm.preprocessing import pad_both_ends
from nltk.metrics.agreement import AnnotationTask
from nltk.tokenize import TweetTokenizer
import difflib as dl
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
from nltk.util import everygrams
from nltk.lm.preprocessing import flatten
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE

### Constants

In [25]:
hate = pd.read_csv("../datasets/hate/train_text.txt", delimiter = "\n", header = None, names = ["tweets"], quoting = 3, skip_blank_lines = False)
hate_test = pd.read_csv("../datasets/hate/test_text.txt", delimiter = "\n", header = None, names = ["tweets"], quoting = 3)
hate_label = pd.read_csv("../datasets/hate/train_labels.txt", delimiter = "\n", header = None, names = ["tweets"], quoting = 3)
hate = [hate["tweets"].isnull() == True]

irony_train_labels = pd.read_csv("../datasets/irony/train_labels.txt", names = ["label"])["label"]
irony_test_labels = pd.read_csv("../datasets/irony/test_labels.txt", names = ["label"])["label"]
irony_train = pd.read_csv("../datasets/irony/train_text.txt", delimiter = "\t", names = ["label"])["label"]
irony_test = pd.read_csv("../datasets/irony/test_text.txt", delimiter = "\t", names = ["label"])["label"]

validation_irony = pd.read_csv("../datasets/irony/val_text.txt", delimiter = "\t", names = ["tweets"])

In [26]:
validation_irony

,tweets
0,#NBA players #NY support protests of #police k...
1,A new year about to start|So many people came ...
2,"Obama's $1,176,120.90 in Taxpayer Funded Cost..."
3,Can't wait to work with the dream team again t...
4,!!! RT @user Of all the places to get stuck in...
...,...
950,Abraham was actually from modern day Iraq (Ur ...
951,@user which one is more disturbing dan? Tickli...
952,@user @user haha that's cool! I had a feeling ...
953,@user @user Let the Western bastards bank acco...


In [27]:
def tokenize_ideal(line):
    tokens = []
    unmatchables = []
    
    for word in line.split():
        if re.findall(r"\w+-\w+|\w+'\w+|\w+|[.&?%!#…]+", word) != []:
            x = re.findall(r"\w+-\w+|\w+'\w+|\w+|[.&?%!#…]+", word)
            for element in x:
                if element == "âž" or element == "ðŸ":
                    continue
                elif element == "&":
                    tokens.append("and")
                else:
                    tokens.append(element.lower())

        if re.findall(r"\w+-\w+|\w+'\w+|\w+|[.&?%!#…]", word) != [word] and re.findall(r"[^\w|.&!?%#…]+", word) != []:
            unmatchables.append(re.findall(r"[^\w|.!#?%…&]+", word)[0])

    s = " ".join(tokens)
    return (s, unmatchables, tokens)

In [28]:
def tokenize_meaning(line):
    tokens = []
    unmatchables = []
    
    for word in line.split():
        if re.findall(r"\w+-\w+|\w+'\w+|\w+|[&?!…]+", word) != []:
            x = re.findall(r"\w+-\w+|\w+'\w+|\w+|[&?!…]+", word)
            for element in x:
                if element == "âž" or element == "ðŸ":
                    continue
                elif element == "&":
                    tokens.append("and")
                else:
                    tokens.append(element.lower())

        if re.findall(r"\w+-\w+|\w+'\w+|\w+|[&?!…]", word) != [word] and re.findall(r"[^\w|&!?…]+", word) != []:
            unmatchables.append(re.findall(r"[^\w|!?…&]+", word)[0])

    s = " ".join(tokens)
    return (s, unmatchables, tokens)

In [29]:
every = []
irony_list = []
irony_string = []
for i in validation_irony["tweets"]:
    #print(i)
    x = tokenize_meaning(i)
    irony_list.append(x[2])
    irony_string.append(x[0])
#print(every)

In [33]:
tknzr = TweetTokenizer()
nltk_tokens = []
nltk_list = []
for tweet in list(validation_irony["tweets"]):
    tokenized = tknzr.tokenize(tweet)
    nltk_list.append(tokenized)
    for token in tokenized:
        nltk_tokens.append(token)

In [53]:
match_ratio = []
for i in range(len(irony_list)):
    match_ratio.append(dl.SequenceMatcher(None, nltk_list[i], irony_list[i]).ratio())
print("Average ratio:",np.mean(match_ratio))    

Average ratio: 0.6440117548786413


In [46]:
s = dl.SequenceMatcher(None, "ab", "AB")
print(s.ratio())

0.0


In [51]:
print(irony_list[6],"\n", nltk_list[6])

['40', 'corner', 'cute', 'day', 'expensive', 'diy', 'crafts', 'please', 'rt'] 
 ['#40', '#Corner', '#Cute', '#Day', '#Expensive', '#diy', '#crafts', '|', 'Please', 'RT', ':']


In [7]:
every = []
list_all = []
for i in hate["tweets"]:
    x = tokenize_meaning(i)
    print(i)
    print(x[0])
    list_all.append(x[0])
    for i in x[0]:
        every.append(i)
#print(every)

@user nice new signage. Are you not concerned by Beatlemania -style hysterical crowds crongregating on you… 
 user nice new signage are you not concerned by beatlemania style hysterical crowds crongregating on you …
A woman who you fucked multiple times saying yo dick small is a compliment you know u hit that spot 😎 
 a woman who you fucked multiple times saying yo dick small is a compliment you know u hit that spot
@user @user real talk do you have eyes or were they gouged out by a rapefugee? 
 user user real talk do you have eyes or were they gouged out by a rapefugee ?
your girlfriend lookin at me like a groupie in this bitch! 
 your girlfriend lookin at me like a groupie in this bitch !
Hysterical woman like @user 
 hysterical woman like user
Me flirting- So tell me about your father... 
 me flirting so tell me about your father
The Philippine Catholic bishops' work for migrant workers should focus on families who are "paying the great... 
 the philippine catholic bishops work for 

You know you're doing something right when you got haters ✌🏼 
 you know you're doing something right when you got haters
@user Massive explosion in Muslim population,illegal migrants r root cause of pollution.Focus on aligning them as good citizens 
 user massive explosion in muslim population illegal migrants r root cause of pollution focus on aligning them as good citizens
Palestinians slam reported US plan to weaken UN 'refugee' agency #UnitedStates... 
 palestinians slam reported us plan to weaken un refugee agency unitedstates
#WorldRefugeeDay is tomorrow! You can help refugee rebuild their lives in the U.S. by joining our World Refugee Day campaign. Get started âž 
 worldrefugeeday is tomorrow ! you can help refugee rebuild their lives in the u s by joining our world refugee day campaign get started
I really do cuss a lot though. Cunt comes out of my mouth all day. Men women and children. No ones safe. 
 i really do cuss a lot though cunt comes out of my mouth all day men women 

AttributeError: 'float' object has no attribute 'split'

In [8]:
w = "that's, a, b"
for i in w.split(","):
    print(re.findall(r"[^a]+", i))
    #print(re.findall(r"\w+", i))

['th', "t's"]
[' ']
[' b']


In [9]:
hate["tokens"] = list_all
hate
len(list_all)

ValueError: Length of values (1467) does not match length of index (9000)

In [10]:
vocabulary = cl.Counter(every)
print(vocabulary)

Counter({' ': 31178, 'e': 16298, 't': 11801, 'a': 11277, 'o': 10455, 'i': 10385, 'n': 9664, 's': 9543, 'r': 9392, 'h': 6195, 'l': 6061, 'u': 5935, 'd': 4615, 'm': 4555, 'c': 4056, 'g': 3713, 'y': 3137, 'w': 2826, 'f': 2770, 'p': 2707, 'b': 2302, 'k': 1533, 'v': 1215, "'": 469, '!': 369, 'j': 282, '?': 236, 'x': 234, '0': 215, '1': 145, 'z': 145, '2': 101, '-': 100, 'q': 69, '3': 67, '…': 56, '4': 51, '5': 51, '6': 39, '7': 38, '8': 38, '9': 21, 'â': 15, 'ð': 12, 'ÿ': 12, '_': 10, 'œ': 6, 'ã': 5, 'é': 2, 'º': 2, 'ª': 2, 'š': 2, 'ï': 2, 'å': 1, 'ä': 1, '²': 1, '³': 1, 'ž': 1, 'ó': 1, '¹': 1})


## N-Grams

In [11]:
text = hate["tokens"]
text

KeyError: 'tokens'

In [12]:
list(nltk.bigrams(text[0]))

NameError: name 'text' is not defined

In [ ]:
list(nltk.bigrams(pad_both_ends(text[0], n=2)))

In [ ]:
def pad(list):
    padded = list(nltk.pad_sequence(text[0], pad_left = True, left_pad_symbol="<s>", pad_right=True, right_pad_symbol="</s>", n=2))
    return padded

In [ ]:
#bigrams
list(nltk.bigrams(pad_both_ends(text[0], n = 2)))

In [ ]:
#Every-grams
from nltk.util import everygrams
padded_bigrams = list(pad_both_ends(text[0], n=2))
list(everygrams(padded_bigrams, max_len=2))

In [ ]:
#vocabulary)
from nltk.lm.preprocessing import flatten
list(flatten(pad_both_ends(sent, n=2) for sent in text))

In [ ]:
def train_ngram(listoflistoftokens, n):
    from nltk.lm.preprocessing import padded_everygram_pipeline
    train, vocab = padded_everygram_pipeline(n, listoflistoftokens)
    
    from nltk.lm import MLE
    lm = MLE(n)
    
    lm.fit(train, vocab)
    
    return lm

In [ ]:
lm3 = train_ngram(hate["tokens"], 3)

In [ ]:
print(lm3.vocab)

In [ ]:
lm3.vocab.lookup(text[0])
#lm3.vocab.lookup(["aliens", "from", "Mars"])

In [ ]:
print(lm3.counts)

In [ ]:
#Scoring
print(lm3.score("alien"))
print(lm3.logscore("the"))
print(lm3.score("bitch", ["stupid"]))

In [ ]:
print(lm3.generate(10))

In [ ]:
lm3.entropy([("stupid", "ass", "bitch"), ("build", "the", "wall")])

In [ ]:
lm3.perplexity([("ass", "stupid")])

In [ ]:
# Trying some Kneser Ney smoothing (not going well)
from nltk.lm.smoothing import KneserNey
from nltk.lm import NgramCounter
from nltk.util import ngrams
ls = [[1,2,3,4],[1,3,3,4]]
mx = []
[[mx.append(num) for num in line] for line in ls]
print(mx)

text_bigrams = []
[[text_bigrams.append(ngrams(sent, 2)) for sent in line] for line in irony_tokens_list]
#text_bigrams = [ngrams(sent, 2) for sent in irony_tokens_list[0]]
text_unigrams = [ngrams(sent, 1) for sent in irony_tokens_list[0]]
#ngram_counts = NgramCounter(text_bigrams + text_unigrams)
ngram_counts = NgramCounter(text_bigrams)

words = irony_tokens_list[0]
from nltk.lm import Vocabulary
vocab = Vocabulary(words, unk_cutoff=2)

x = KneserNey(vocab, ngram_counts)
x.alpha_gamma("ppl", irony_tokens_list[0])

## Annotations for irony dataset

In [ ]:
rasmus_labels = pd.read_csv("../datasets/annotations/annotations_rasmus.csv", names = ["label"])
ruben_labels = pd.read_csv("../datasets/annotations/annotations_ruben.csv", names = ["label"])
sabrina_labels = pd.read_csv("../datasets/annotations/annotations_sabrina.csv", names = ["label"])
ida_labels = pd.read_csv("../datasets/annotations/annotations_ida.csv", names = ["label"])
true_labels = pd.read_csv("../datasets/annotations/iaa_labels.txt", names = ["label"])

df = pd.DataFrame()

df["sabrina"] = sabrina_labels["label"]
df["rasmus"] = rasmus_labels["label"]
df["ruben"] = ruben_labels["label"]
df["ida"] = ida_labels["label"]

df["truth"] = true_labels["label"]

df

In [ ]:
for name in df.columns:
    score = 0
    for i in range(len(df[name])):
        if list(df[name])[i] == list(df["truth"])[i]:
            score += 1
    print(name, score/120)

In [ ]:
rasmus_task = [("rasmus", i, list(df["rasmus"])[i]) for i in range(len(list(df["rasmus"])))]
sabrina_task = [("sabrina", i, list(df["sabrina"])[i]) for i in range(len(list(df["sabrina"])))]
ruben_task = [("ruben", i, list(df["ruben"])[i]) for i in range(len(list(df["ruben"])))]
ida_task = [("ida", i, list(df["ida"])[i]) for i in range(len(list(df["ida"])))]
truth_task = [("truth", i, list(df["truth"])[i]) for i in range(len(list(df["truth"])))]

ls = []
for i in range(len(rasmus_task)):
    ls.append(rasmus_task[i])
    ls.append(sabrina_task[i])
    ls.append(ruben_task[i])
    ls.append(ida_task[i])
    #ls.append(truth_task[i])

t = AnnotationTask(data=ls)
t.avg_Ao()

In [ ]:
print(t.pi())
print(t.kappa())

# Creating bag of words feature

In [15]:
test_vocab

[' user nice new signage are you not concerned by beatlemania style hysterical crowds crongregating on you …',
 ' a woman who you fucked multiple times saying yo dick small is a compliment you know u hit that spot',
 ' user user real talk do you have eyes or were they gouged out by a rapefugee ?',
 ' your girlfriend lookin at me like a groupie in this bitch !',
 ' hysterical woman like user',
 ' me flirting so tell me about your father',
 ' the philippine catholic bishops work for migrant workers should focus on families who are paying the great',
 " i am not going after your ex bf you lieing sack of shit ! i'm done with you dude that's why i dumped your ass cause your a lieing bitch",
 ' when cuffin season is finally over',
 ' send home migrants not in need of protection peter dutton tells un hey dutton how about the ones that have stayed and not left the country when they should overstayers ? why dont you go and round all them up ?',
 ' cory booker and kamala harris competing for mos

### SGDClassifier for the hate dataset without pipelining

In [16]:
hate_train_vocab = []
hate_test_vocab = []
for i in hate["tweets"]:
    try:
        x = tokenize_meaning(i)
        hate_train_vocab.append(x[0])
    except:
        hate_train_vocab.append("")

for i in hate_test["tweets"]:
    x = tokenize_meaning(i)
    hate_test_vocab.append(x[0])

TypeError: list indices must be integers or slices, not str

In [ ]:
hate_train_vocab[1467]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(analyzer='word', ngram_range=(2, 2))
X_train_counts = count_vect.fit_transform(hate_train_vocab)
X_train_counts.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

In [ ]:
labels = pd.read_csv("../datasets/hate/train_labels.txt", names = ["label"])["label"]
test_labels = pd.read_csv("../datasets/hate/test_labels.txt", names = ["label"])["label"]
print(len(hate_train_vocab))
print(len(labels))
print(len(hate_test_vocab))
print(len(test_labels))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, labels)

Doing everything in one line with pipelining

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', SGDClassifier(loss='log'))])
text_clf.fit(hate_train_vocab, labels)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
predicted = text_clf.predict(hate_test_vocab)
print(np.mean(predicted == test_labels))
#accuracy_score(predicted, test_labels[0:2932])

## SGDClassifier for the irony dataset with pipelining

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier

In [1]:
irony_train_labels = pd.read_csv("../datasets/irony/train_labels.txt", names = ["label"])["label"]
irony_test_labels = pd.read_csv("../datasets/irony/test_labels.txt", names = ["label"])["label"]
irony_train = pd.read_csv("../datasets/irony/train_text.txt", delimiter = "\t", names = ["label"])["label"]
irony_test = pd.read_csv("../datasets/irony/test_text.txt", delimiter = "\t", names = ["label"])["label"]

NameError: name 'pd' is not defined

In [25]:
irony_train_vocab = []
irony_train_list = []
irony_test_vocab = []
for i in irony_train:
    x = tokenize_meaning(i)
    irony_train_vocab.append(x[0])
    irony_train_list.append(x[2])

for i in irony_test:
    x = tokenize_meaning(i)
    irony_test_vocab.append(x[0])

https://www.shanelynn.ie/get-busy-with-word-embeddings-introduction/ <br>
https://www.shanelynn.ie/word-embeddings-in-python-with-spacy-and-gensim/ <br>

## Pre trained model with Spacy

In [39]:
import spacy

# Load the spacy model that you have installed
nlp = spacy.load('en_core_web_md')

# process a sentence using the model
doc = nlp("This is some text that I am processing with Spacy")

# It's that simple - all of the vectors and words are assigned after this point
# Get the vector for 'text':
#doc[3].vector

# Get the mean vector for the entire sentence (useful for sentence classification etc.)
doc.vector

array([-5.36412969e-02,  2.79353321e-01, -1.05259977e-01, -1.76284965e-02,
        1.34550199e-01,  1.92671806e-01,  5.50469756e-03, -2.39132687e-01,
       -4.06342074e-02,  1.78010297e+00, -1.80772960e-01,  1.02661893e-01,
        6.84069991e-02, -5.09319194e-02, -7.65837058e-02, -3.77540514e-02,
        8.24129581e-03,  1.37752008e+00, -1.78934380e-01, -5.76109104e-02,
        1.66338980e-02, -3.62196006e-02, -7.48579949e-02,  4.40651290e-02,
       -2.65241470e-02,  2.41529979e-02,  9.79370065e-03, -1.13990309e-03,
        1.59522101e-01, -1.56648397e-01, -9.12139937e-02,  9.11872908e-02,
        1.07169405e-01, -1.08843103e-01, -7.94988051e-02, -4.74919155e-02,
       -1.60613850e-01, -2.82304995e-02, -1.03425637e-01, -1.14933215e-01,
        1.62531182e-01, -1.01342008e-01,  2.17013666e-03,  3.47881988e-02,
       -6.34927005e-02,  2.44374484e-01, -3.01910043e-02, -1.46046979e-02,
       -1.06488302e-01,  6.26319647e-03, -1.30655810e-01,  7.04905912e-02,
       -4.86716032e-02,  

## Pre trained model with Gensim

In [76]:
from gensim.models import KeyedVectors

# Load vectors directly from the file
model = KeyedVectors.load_word2vec_format('../datasets/GoogleNews-vectors-negative300.bin', binary=True)

# Access vectors for specific words with a keyed lookup:
vector = model['easy']
# see the shape of the vector (300,)
vector.shape

# Processing sentences is not as simple as with Spacy:
vectors = [model[x] for x in "This is some text I am processing with Spacy".split(' ')]

In [109]:
model.most_similar("good")

[('great', 0.7291510105133057),
 ('bad', 0.7190050482749939),
 ('terrific', 0.6889114379882812),
 ('decent', 0.6837348937988281),
 ('nice', 0.6836092472076416),
 ('excellent', 0.6442928910255432),
 ('fantastic', 0.6407778263092041),
 ('better', 0.6120728850364685),
 ('solid', 0.5806034803390503),
 ('lousy', 0.5764201879501343)]

## Custom word embedding 

In [41]:
import os
import sys
import re
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser, Phrases

In [106]:
all_sentences = irony_train_list
common_terms = ["of", "with", "without", "and", "or", "the", "a", "it", "it's", "its"]
# Create the relevant phrases from the list of sentences:
phrases = Phrases(all_sentences, connector_words = common_terms)
# The Phraser object is used from now on to transform sentences
bigram = Phraser(phrases)
# Applying the Phraser to transform our sentences is simply
all_sentences = list(bigram[all_sentences])

In [107]:
model_custom = Word2Vec(all_sentences, 
                 min_count=2,   # Ignore words that appear less than this
                 vector_size=50,      # Dimensionality of word embeddings
                 workers=2,     # Number of processors (parallelisation)
                 window=3,      # Context window for words during training
                 epochs=30)       # Number of epochs training over corpus

In [108]:
model_custom.wv.most_similar("good", topn = 1000)

[('such', 0.944770097732544),
 ('its', 0.9300004839897156),
 ('gonna_be', 0.9230743646621704),
 ('smile', 0.9225327372550964),
 ('game', 0.91889888048172),
 ('today', 0.9173682332038879),
 ('lovely', 0.914467453956604),
 ('lot', 0.9129676222801208),
 ("it's", 0.9105861783027649),
 ('leader', 0.8995773792266846),
 ('doing', 0.8987858891487122),
 ('bit', 0.8984308838844299),
 ('diane', 0.8970010876655579),
 ('cthulhu', 0.8960486650466919),
 ("you're", 0.8955676555633545),
 ('seasons', 0.8948409557342529),
 ('banter', 0.8947873115539551),
 ('should_be', 0.8922043442726135),
 ('bad', 0.8916842937469482),
 ('luck', 0.8916140198707581),
 ('end', 0.8912337422370911),
 ('movie', 0.8911839127540588),
 ('e', 0.8905121684074402),
 ('exciting', 0.8900037407875061),
 ('surprise', 0.8894014358520508),
 ('2015', 0.889264702796936),
 ('joke', 0.8876233696937561),
 ('customer', 0.8871283531188965),
 ('called', 0.886785089969635),
 ('generous', 0.8867736458778381),
 ('insight', 0.8844496011734009),
 ('f

In [99]:
model_custom.wv.vectors.shape[0]

2911

In [105]:
?Phrases

In [31]:
#for testing hate dataset
# irony_train_vocab = hate_train_vocab
# irony_test_vocab = hate_test_vocab
# irony_train_labels = labels[0:8993]
# irony_test_labels = test_labels
# print(len(irony_test_vocab))
irony_train_list

[['seeing',
  'ppl',
  'walking',
  'w',
  'crutches',
  'makes',
  'me',
  'really',
  'excited',
  'for',
  'the',
  'next',
  '3',
  'weeks',
  'of',
  'my',
  'life'],
 ['look',
  'for',
  'the',
  'girl',
  'with',
  'the',
  'broken',
  'smile',
  'ask',
  'her',
  'if',
  'she',
  'wants',
  'to',
  'stay',
  'while',
  'and',
  'she',
  'will',
  'be',
  'loved'],
 ['now',
  'i',
  'remember',
  'why',
  'i',
  'buy',
  'books',
  'online',
  'user',
  'servicewithasmile'],
 ['user',
  'user',
  'so',
  'is',
  'he',
  'banded',
  'from',
  'wearing',
  'the',
  'clothes',
  '?',
  'karma'],
 ['just',
  'found',
  'out',
  'there',
  'are',
  'etch',
  'a',
  'sketch',
  'apps',
  'oldschool',
  'notoldschool'],
 ['hey',
  'what',
  'do',
  'you',
  'know',
  'one',
  'of',
  'the',
  'witnesses',
  'supporting',
  'darren',
  "wilson's",
  'story',
  'lied',
  '!',
  'and',
  'is',
  'racist',
  '!',
  'mind',
  'blown',
  '!'],
 ['user',
  'on',
  'stage',
  'at',
  'flzjingl

In [21]:
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html
for i in ["hinge", "log", "modified_huber", "squared_hinge", "perceptron", "squared_loss", "huber", "epsilon_insensitive", "squared_epsilon_insensitive"]:
    irony_sgdc = Pipeline([('vect', CountVectorizer(analyzer = 'word', ngram_range=(1, 5))), ('tfidf', TfidfTransformer()), ('clf', SGDClassifier(loss = i))])
    irony_sgdc.fit(irony_train_vocab, irony_train_labels)
    predicted = irony_sgdc.predict(irony_test_vocab)
    print(i, np.mean(predicted == irony_test_labels))

hinge 0.6543367346938775
log 0.6505102040816326
modified_huber 0.6556122448979592
squared_hinge 0.6658163265306123
perceptron 0.6479591836734694
squared_loss 0.6466836734693877
huber 0.5688775510204082
epsilon_insensitive 0.6556122448979592
squared_epsilon_insensitive 0.4528061224489796


Brown clustering: https://github.com/percyliang/brown-cluster
Word embedding

ngram_range(1,2)
Kneser-Ney smoothing for the ngram
Witten-Bell smoothing for the ngram

## Support Vector Machine

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
from sklearn.svm import SVC
for i in ['linear', 'poly', 'rbf', 'sigmoid']:
    irony_svc = Pipeline([('vect', CountVectorizer(analyzer='word', ngram_range=(1, 2))), ('tfidf', TfidfTransformer()), ('clf', SVC(gamma='scale', degree = 2, kernel = i))])
    irony_svc.fit(irony_train_vocab, irony_train_labels)
    predicted = irony_svc.predict(irony_test_vocab)
    print(i, np.mean(predicted == irony_test_labels))

## Naive_bayes

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html
from sklearn.naive_bayes import MultinomialNB
irony_bay = Pipeline([('vect', CountVectorizer(analyzer='word', ngram_range=(1, 2))), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
irony_bay.fit(irony_train_vocab, irony_train_labels)
predicted = irony_bay.predict(irony_test_vocab)
print(np.mean(predicted == irony_test_labels))

## Multi-Layer Perceptron

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
from sklearn.neural_network import MLPClassifier
irony_multi = Pipeline([('vect', CountVectorizer(analyzer='word', ngram_range=(1, 2))), ('tfidf', TfidfTransformer()), ('clf', MLPClassifier(random_state=1, max_iter=300))])
irony_multi.fit(irony_train_vocab, irony_train_labels)
predicted = irony_multi.predict(irony_test_vocab)
print(np.mean(predicted == irony_test_labels))

## Decision tree

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
from sklearn.tree import DecisionTreeClassifier
irony_tree = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', DecisionTreeClassifier(random_state=0))])
irony_tree.fit(irony_train_vocab, irony_train_labels)
predicted = irony_tree.predict(irony_test_vocab)
print(np.mean(predicted == irony_test_labels))

## Random forest

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
from sklearn.ensemble import RandomForestClassifier
irony_forest = Pipeline([('vect', CountVectorizer(analyzer='word', ngram_range=(1, 1))), ('tfidf', TfidfTransformer()), ('clf', RandomForestClassifier())])
irony_forest.fit(irony_train_vocab, irony_train_labels)
predicted = irony_forest.predict(irony_test_vocab)
print(np.mean(predicted == irony_test_labels))

In [54]:
def max_likelihood(tokenlist):
    padded_bigrams = list(pad_both_ends(tokenlist, n=2))
    ex2 = list(everygrams(padded_bigrams, max_len=2))
    ex3 = list(everygrams(padded_bigrams, max_len=3))
    #print(ex2, "\n")
    #print(ex3)

    unigram_tokens = {}
    bigram_tokens = {}

    count = 0
    for i in range(len(ex2)):
        for j in range(len(ex2)):
            if len(ex2[i]) == 1: 
                if ex2[i] == ex2[j]:
                    count += 1
                    unigram_tokens[ex2[i]] = count
            elif len(ex2[i]) == 2:
                if ex2[i] == ex2[j]:
                    count += 1
                    bigram_tokens[ex2[i]] = count
        count = 0

    scores = {}
    for unitoken in unigram_tokens:
        for bitoken in bigram_tokens:
            if unitoken[0] in bitoken:
                p = bigram_tokens[bitoken]/unigram_tokens[unitoken]
                scores[bitoken[1]+"|"+bitoken[0]] = p

    return scores

In [ ]:
for tweet in train_irony["tweets"]:
    tokenized = tokenize_meaning(tweet)
    max_like = max_likelihood(tokenized[2])
    for key,val in zip(max_like.keys(),max_like.values()):
        print((key,val))